## How to obtain more info from TTree::Draw

### 1. to TH

- 按照ROOT默认的bin和范围画图
``` cpp
tree->Draw("counts");        
tree->Draw("counts>>h");     // 生成以"h"为名的histogram 
tree->Draw("counts >>+ h")   // 如果已存在"h"为名的histogram，将数据追加到“h”上
``` 
- 指定histogram
``` cpp
tree->Draw("counts>>h(100, 0, 100)"    // nbin,xmin,xmax
tree->Draw("counts>>h(100, 0, )"       // nbin,xmin,不指定xmax
tree->Draw("counts>>h(100, ,100 )"       // nbin,不指定xmin,xmax  
```
- 先定义histogram
``` cpp
TH1D *h1 = new TH1D("hoge", "hoge", 100, 0, 100);
tree->Draw("counts>>hoge");     
```
- TH2,TH3 以此类推

### 2. to TGraph
- Once TTree::Draw has been called, it is possible to access useful information still stored in the TTree object via the following functions:
``` cpp
    -GetSelectedRows()    // return the number of entries accepted by the
                          //selection expression. In case where no selection
                          //was specified, returns the number of entries processed.
    -GetV1()              //returns a pointer to the float array of V1
    -GetV2()              //returns a pointer to the float array of V2
    -GetV3()              //returns a pointer to the float array of V3
    -GetW()               //returns a pointer to the double array of Weights
                          //where weight equal the result of the selection expression.
```
   where V1,V2,V3 correspond to the expressions in
``` cpp   
   TTree::Draw("V1:V2:V3",selection);
```
   Example:
``` cpp
    tree->Draw("py:px","pz>4");
    TGraph *gr = new TGraph(tree->GetSelectedRows(),ntuple->GetV2(), ntuple->GetV1());
    gr->Draw("ap"); //draw graph in current pad creates a TGraph object with a number of points corresponding to the number of entries selected by the expression "pz>4", the x points of the graph being the px values of the Tree and the y points the py values.
``` 

   Important note: By default TTree::Draw creates the arrays obtained with GetV1, GetV2, GetV3, GetW with a length corresponding to the
    parameter fEstimate. By default fEstimate=10000 and can be modified via TTree::SetEstimate. 
    A possible recipee is to do
``` cpp
    tree->SetEstimate(tree->GetEntries());
```
You must call SetEstimate if the expected number of selected rows is greater than 10000.

You can use the option "goff" to turn off the graphics output of TTree::Draw in the above example.

### 3. to TVectorD

``` cpp
tree->Draw("x:y",tcut); // alternative: tr->Project("", "x:y", tcut);    
TVectorD *x_vec = new TVectorD(tree->GetSelectedRows(), tr->GetV1());    
TVectorD *y_vec = new TVectorD(tree->GetSelectedRows(), tr->GetV2());
``` 



## How to Use Chains (Lists of Files)?

TChain object is a list of Root files containing the same tree. As an example, assume we have 3 Root files called file1.root, file2.root, file3.root. Each file contains one tree called "T". We can create a chain with the following statements:
``` cpp
   TChain chain("T");
   chain.Add("file1.root");
   chain.Add("file2.root");
   chain.Add("file3.root");
``` 
The class TChain is derived from the class TTree. For example, to generate an histogram corresponding to the attribute "x" in tree "T" by processing sequentially the 3 files of this chain, we can do:
``` cpp
   chain.Draw("x");
``` 

Chain files with loop
``` cpp
TChain *chain=new TChain("chain", "chainname");
const Int_t fNFile=11;
Int_t iFile;
for (iFile=0; iFile<fNFile; ++iFile) {
    chain->Add(Form("../anadata/CALIB_RUN%d.root", iFile+10));
}
``` 


## Convert TGraph *g to TH2D *h
``` cpp
int n=g->GetN();
double *x=g->GetX();
double *y=g->GetY();
for(int i=0;i<n;i++) h->Fill(x,y);
h->Draw("colz")
``` 

## Adding points to a graph
- Add point (x,y) to a grahp g
``` cpp
g->SetPoint(g->GetN(), x,y)
``` 

## 在代码中运行脚本
``` cpp
gROOT->ProcessLine(".L anotherScript.c");
``` 